In [2]:
%pip install -U langchain-community langchain-ollama langchain-qdrant

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1")

In [4]:
llm.invoke("What is LangChain?")

'LangChain is an open-source, modular, and extensible framework for building conversational AI models, particularly those based on large language models (LLMs) like transformers. It was created by Meta AI to simplify the development of conversational interfaces that can have human-like conversations with users.\n\nThe name "LangChain" suggests a connection between languages or chains of thought, emphasizing the idea of facilitating meaningful interactions through natural language processing and generation.\n\nSome key features and goals of LangChain include:\n\n1. **Modularity**: Breakdown complex conversational AI systems into smaller, reusable components that can be easily combined.\n2. **Extensibility**: Allow developers to add new capabilities or models as needed, making the framework more adaptable to evolving use cases.\n3. **Simplification**: Reduce the complexity of building and integrating conversational AI models, making it easier for both beginners and experienced developers

In [5]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT_TEMPLATE = """\
<|start_header_id|>system<|end_header_id|>
You are a helpful assistant. You answer user questions based on provided context. If you can't answer the question with the provided context, say you don't know.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
User Query:
{query}

Context:
{context}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [6]:
rag_chain = rag_prompt | llm

In [7]:
rag_chain.invoke({"query" : "Who old is Carl?", "context" : "Carl is a sweet dude, he's 40."})

'Carl is 40 years old!'

In [8]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [9]:
from langchain_community.document_loaders import TextLoader

document_loader = TextLoader("./data/paul_graham_essays.txt")
documents = document_loader.load()

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
split_documents = text_splitter.split_documents(documents)
len(split_documents)

4265

In [39]:
from langchain_qdrant import QdrantVectorStore

url = "localhost:6333"
qdrant = QdrantVectorStore.from_documents(
    split_documents,
    embeddings,
    url=url,
    prefer_grpc=True,
    collection_name="Paul_Graham_Documents",
)

In [12]:
from langchain_qdrant import QdrantVectorStore

url = "localhost:6333"
qdrant_vectorstore = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="Paul_Graham_Documents",
    prefer_grpc=True,
    url=url
    )